In [47]:
#DataFrame are RDD with schema on it.
#It can be created by various method
#[reading textdata from files]
#[reading data from Hive , mysql over jdbc, +more]
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('DF Introduction').getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", '/user/hive/warehouse') \
    .enableHiveSupport() \
    .getOrCreate()


In [229]:
#orders = spark.read.table('orders')
#orders = spark.sql('select * from orders')

In [54]:
df1 = spark.read.csv('/home/hduser/Documents/RddTransformations/Data/orders.csv')
df1.printSchema()
df1.toDF()
#default it will create it's own schema string as default data type and _c0, _c1 as default column name

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [101]:
df1 = spark.read.csv('/home/hduser/Documents/RddTransformations/Data/orders.csv').toDF('order_id','order_date','customer_id','order_status')
#df1 = spark.read.csv('/home/hduser/Documents/RddTransformations/Data/orders.csv',inferSchema= True).toDF('order_id','order_date','customer_id','order_status')

df1.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [56]:
df1.select('order_id','order_date').show()

+--------+-------------------+
|order_id|         order_date|
+--------+-------------------+
|       1|2013-07-25 00:00:00|
|       2|2013-07-25 00:00:00|
|       3|2013-07-25 00:00:00|
|       4|2013-07-25 00:00:00|
|       5|2013-07-25 00:00:00|
|       6|2013-07-25 00:00:00|
|       7|2013-07-25 00:00:00|
|       8|2013-07-25 00:00:00|
|       9|2013-07-25 00:00:00|
|      10|2013-07-25 00:00:00|
|      11|2013-07-25 00:00:00|
|      12|2013-07-25 00:00:00|
|      13|2013-07-25 00:00:00|
|      14|2013-07-25 00:00:00|
|      15|2013-07-25 00:00:00|
|      16|2013-07-25 00:00:00|
|      17|2013-07-25 00:00:00|
|      18|2013-07-25 00:00:00|
|      19|2013-07-25 00:00:00|
|      20|2013-07-25 00:00:00|
+--------+-------------------+
only showing top 20 rows



In [57]:
df1.describe()

DataFrame[summary: string, order_id: string, order_date: string, customer_id: string, order_status: string]

In [58]:
df1.describe().show()
#give the metris about the data

+-------+------------------+-------------------+-----------------+---------------+
|summary|          order_id|         order_date|      customer_id|   order_status|
+-------+------------------+-------------------+-----------------+---------------+
|  count|             68883|              68883|            68883|          68883|
|   mean|           34442.0|               null|6216.571098819738|           null|
| stddev|19884.953633337947|               null|3586.205241263963|           null|
|    min|                 1|2013-07-25 00:00:00|                1|       CANCELED|
|    max|              9999|2014-07-24 00:00:00|             9999|SUSPECTED_FRAUD|
+-------+------------------+-------------------+-----------------+---------------+



In [230]:
#How to change the data types of columns in dataFrame. 
#df1 all columns are string but order_id and customer_id are integer. We can change this using 
#withColumn("column_name", df.column_name.cast(DoubleType())
from pyspark.sql.types import IntegerType
#dftyped = df1. \
#            withColumn('order_id', df1.order_id.cast(IntegerType())). \
#            withColumn('customer_id', df1.customer_id.cast(IntegerType()))

#or we can predefine schema and apply while reading

schema = StructType([StructField("order_id", IntegerType()),StructField("order_date", StringType()), StructField("customer_id", IntegerType()),StructField("order_status", StringType())])
# Finally, we create the dataframe with previous variables
# Also we can specifying the separator with 'sep' (default separator for CSV is ',')
orderDF = spark.read.csv(path='/home/hduser/Documents/RddTransformations/Data/orders.csv', sep=",", schema=schema)

In [82]:
#since now two columns are integertype and DataTypes are in pyspark.sql.types library
dftyped.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [102]:
#Read data from hive spark.read.table(dbname.table)
#order_items = spark.read.table('order_items')

In [104]:
from pyspark.sql.functions import *
#These can be categorised into String manipulation, Date manipulation, Type casting, Expression such as case when

In [61]:
dftyped.show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
|       6|2013-07-25 00:00:00|       7130|       COMPLETE|
|       7|2013-07-25 00:00:00|       4530|       COMPLETE|
|       8|2013-07-25 00:00:00|       2911|     PROCESSING|
|       9|2013-07-25 00:00:00|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSIN

In [64]:
dftyped.select(substring(dftyped.order_date, 1,7).alias('orderDate')).show()

+---------+
|orderDate|
+---------+
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
|  2013-07|
+---------+
only showing top 20 rows



In [72]:
dftyped.select(lower(dftyped.order_status).alias('order_status')).show()
#it will only show that columnwith select function

+---------------+
|   order_status|
+---------------+
|         closed|
|pending_payment|
|       complete|
|         closed|
|       complete|
|       complete|
|       complete|
|     processing|
|pending_payment|
|pending_payment|
| payment_review|
|         closed|
|pending_payment|
|     processing|
|       complete|
|pending_payment|
|       complete|
|         closed|
|pending_payment|
|     processing|
+---------------+
only showing top 20 rows



In [71]:
dftyped.withColumn('order_status', lower(dftyped.order_status)).show()

#withColumn will show all the other columns along with the processed column

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         closed|
|       2|2013-07-25 00:00:00|        256|pending_payment|
|       3|2013-07-25 00:00:00|      12111|       complete|
|       4|2013-07-25 00:00:00|       8827|         closed|
|       5|2013-07-25 00:00:00|      11318|       complete|
|       6|2013-07-25 00:00:00|       7130|       complete|
|       7|2013-07-25 00:00:00|       4530|       complete|
|       8|2013-07-25 00:00:00|       2911|     processing|
|       9|2013-07-25 00:00:00|       5657|pending_payment|
|      10|2013-07-25 00:00:00|       5648|pending_payment|
|      11|2013-07-25 00:00:00|        918| payment_review|
|      12|2013-07-25 00:00:00|       1837|         closed|
|      13|2013-07-25 00:00:00|       9149|pending_payment|
|      14|2013-07-25 00:00:00|       9842|     processin

In [75]:
dftyped.withColumn('order_date', date_format('order_date', 'yyyyMM')).show()
#using multiple withColumn is also allowed for multiple column transformations
#dftyped.withColumn('order_date', date_format('order_date', 'yyyyMM')).withColumn('order_status', lower(dftyped.order_status)).show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|    201307|      11599|         closed|
|       2|    201307|        256|pending_payment|
|       3|    201307|      12111|       complete|
|       4|    201307|       8827|         closed|
|       5|    201307|      11318|       complete|
|       6|    201307|       7130|       complete|
|       7|    201307|       4530|       complete|
|       8|    201307|       2911|     processing|
|       9|    201307|       5657|pending_payment|
|      10|    201307|       5648|pending_payment|
|      11|    201307|        918| payment_review|
|      12|    201307|       1837|         closed|
|      13|    201307|       9149|pending_payment|
|      14|    201307|       9842|     processing|
|      15|    201307|       2568|       complete|
|      16|    201307|       7276|pending_payment|
|      17|    201307|       2667|       complete|


In [108]:
dftyped.selectExpr('date_format(order_date,"yyyMM") as order_month').show()

+-----------+
|order_month|
+-----------+
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
|     201307|
+-----------+
only showing top 20 rows



In [137]:
#filtering rows from DF
#dftyped.where(dftyped.order_status == "COMPLETE").show()
#dftyped.where('order_status = "COMPLETE"').show()
#dftyped.where('order_status in ("COMPLETE","CLOSED"').show()
dftyped.where((dftyped.order_status == "COMPLETE").__or__(dftyped.order_status == "CLOSED")).show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|       1|2013-07-25 00:00:00|      11599|      CLOSED|
|       3|2013-07-25 00:00:00|      12111|    COMPLETE|
|       4|2013-07-25 00:00:00|       8827|      CLOSED|
|       5|2013-07-25 00:00:00|      11318|    COMPLETE|
|       6|2013-07-25 00:00:00|       7130|    COMPLETE|
|       7|2013-07-25 00:00:00|       4530|    COMPLETE|
|      12|2013-07-25 00:00:00|       1837|      CLOSED|
|      15|2013-07-25 00:00:00|       2568|    COMPLETE|
|      17|2013-07-25 00:00:00|       2667|    COMPLETE|
|      18|2013-07-25 00:00:00|       1205|      CLOSED|
|      22|2013-07-25 00:00:00|        333|    COMPLETE|
|      24|2013-07-25 00:00:00|      11441|      CLOSED|
|      25|2013-07-25 00:00:00|       9503|      CLOSED|
|      26|2013-07-25 00:00:00|       7562|    COMPLETE|
|      28|2013-07-25 00:00:00|        656|    CO

In [139]:
#dftyped.filter(dftyped.order_status == "COMPLETE").show()
#dftyped.filter('order_status = "COMPLETE"').show()
#dftyped.filter('order_status in ("COMPLETE","CLOSED")').show()
dftyped.filter(dftyped.order_status.isin("COMPLETE","CLOSED")).show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|       1|2013-07-25 00:00:00|      11599|      CLOSED|
|       3|2013-07-25 00:00:00|      12111|    COMPLETE|
|       4|2013-07-25 00:00:00|       8827|      CLOSED|
|       5|2013-07-25 00:00:00|      11318|    COMPLETE|
|       6|2013-07-25 00:00:00|       7130|    COMPLETE|
|       7|2013-07-25 00:00:00|       4530|    COMPLETE|
|      12|2013-07-25 00:00:00|       1837|      CLOSED|
|      15|2013-07-25 00:00:00|       2568|    COMPLETE|
|      17|2013-07-25 00:00:00|       2667|    COMPLETE|
|      18|2013-07-25 00:00:00|       1205|      CLOSED|
|      22|2013-07-25 00:00:00|        333|    COMPLETE|
|      24|2013-07-25 00:00:00|      11441|      CLOSED|
|      25|2013-07-25 00:00:00|       9503|      CLOSED|
|      26|2013-07-25 00:00:00|       7562|    COMPLETE|
|      28|2013-07-25 00:00:00|        656|    CO

In [147]:
#get the records which order_status are COMPLETE and CLOSED and order_date 2013 Aug
dftyped.filter(dftyped.order_status.isin("COMPLETE","CLOSED").__and__(date_format(dftyped.order_date,"yyyMM") == "201308")).show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|    1297|2013-08-01 00:00:00|      11607|    COMPLETE|
|    1298|2013-08-01 00:00:00|       5105|      CLOSED|
|    1299|2013-08-01 00:00:00|       7802|    COMPLETE|
|    1302|2013-08-01 00:00:00|       1695|    COMPLETE|
|    1304|2013-08-01 00:00:00|       2059|    COMPLETE|
|    1305|2013-08-01 00:00:00|       3844|    COMPLETE|
|    1307|2013-08-01 00:00:00|       4474|    COMPLETE|
|    1309|2013-08-01 00:00:00|       2367|      CLOSED|
|    1312|2013-08-01 00:00:00|      12291|    COMPLETE|
|    1314|2013-08-01 00:00:00|      10993|    COMPLETE|
|    1315|2013-08-01 00:00:00|       5660|    COMPLETE|
|    1318|2013-08-01 00:00:00|       4212|    COMPLETE|
|    1319|2013-08-01 00:00:00|       3966|    COMPLETE|
|    1320|2013-08-01 00:00:00|      12270|    COMPLETE|
|    1321|2013-08-01 00:00:00|        800|    CO

In [148]:
#Joins using DF.
#df1.join(df2, columnforJoin) #if both the df has same column
#df1.join(df2,df1.col1 == df2.col2, how[`inner``, ``cross``, ``outer``,
#    ``full``, ``full_outer``, ``left``, ``left_outer``, ``right``, ``right_outer``,
#    ``left_semi``, and ``left_anti``]) # if both the df has different join columns

dftyped.filter(dftyped.order_status.isin("COMPLETE","CLOSED")).show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|       1|2013-07-25 00:00:00|      11599|      CLOSED|
|       3|2013-07-25 00:00:00|      12111|    COMPLETE|
|       4|2013-07-25 00:00:00|       8827|      CLOSED|
|       5|2013-07-25 00:00:00|      11318|    COMPLETE|
|       6|2013-07-25 00:00:00|       7130|    COMPLETE|
|       7|2013-07-25 00:00:00|       4530|    COMPLETE|
|      12|2013-07-25 00:00:00|       1837|      CLOSED|
|      15|2013-07-25 00:00:00|       2568|    COMPLETE|
|      17|2013-07-25 00:00:00|       2667|    COMPLETE|
|      18|2013-07-25 00:00:00|       1205|      CLOSED|
|      22|2013-07-25 00:00:00|        333|    COMPLETE|
|      24|2013-07-25 00:00:00|      11441|      CLOSED|
|      25|2013-07-25 00:00:00|       9503|      CLOSED|
|      26|2013-07-25 00:00:00|       7562|    COMPLETE|
|      28|2013-07-25 00:00:00|        656|    CO

In [150]:
#Grouping and Aggregation [join, groupBy, sort or orderBy]
#dftyped.join()

In [151]:

from pyspark.sql.types import * 
schema = StructType([StructField("order_item_id", IntegerType()),StructField("order_item_order_id", IntegerType()),StructField("order_item_product_id", IntegerType()),StructField("order_item_quantity", IntegerType()),StructField("order_item_subtotal", FloatType()), StructField("order_item_product_price", FloatType())])
orderItemsDF = spark.read.csv(path='/home/hduser/Documents/RddTransformations/Data/order_items.csv', sep=",", schema=schema)

In [152]:
orderItemsDF.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [156]:
#get the sum of revenue particular order
#orderItemsDF.filter(orderItemsDF.order_item_order_id == 2).agg(sum('order_item_subtotal')).show()
orderItemsDF.groupBy('order_item_order_id').sum('order_item_subtotal').alias("sum").show()

+-------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|
+-------------------+------------------------+
|                148|      479.99000549316406|
|                463|       829.9200096130371|
|                471|      169.98000717163086|
|                496|        441.950008392334|
|               1088|      249.97000885009766|
|               1580|      299.95001220703125|
|               1591|       439.8599967956543|
|               1645|       1509.790023803711|
|               2366|       299.9700012207031|
|               2659|       724.9100151062012|
|               2866|        569.960018157959|
|               3175|      209.97000122070312|
|               3749|      143.97000122070312|
|               3794|      299.95001220703125|
|               3918|       829.9300155639648|
|               3997|       579.9500122070312|
|               4101|      129.99000549316406|
|               4519|        79.9800033569336|
|            

In [157]:
#Sort by Order_status
dftyped.orderBy('order_status').show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|     527|2013-07-28 00:00:00|       5426|    CANCELED|
|    1435|2013-08-01 00:00:00|       1879|    CANCELED|
|     552|2013-07-28 00:00:00|       1445|    CANCELED|
|     112|2013-07-26 00:00:00|       5375|    CANCELED|
|     564|2013-07-28 00:00:00|       2216|    CANCELED|
|     955|2013-07-30 00:00:00|       8117|    CANCELED|
|    1383|2013-08-01 00:00:00|       1753|    CANCELED|
|     962|2013-07-30 00:00:00|       9492|    CANCELED|
|     607|2013-07-28 00:00:00|       6376|    CANCELED|
|    1013|2013-07-30 00:00:00|       1903|    CANCELED|
|     667|2013-07-28 00:00:00|       4726|    CANCELED|
|    1169|2013-07-31 00:00:00|       3971|    CANCELED|
|     717|2013-07-29 00:00:00|       8208|    CANCELED|
|    1186|2013-07-31 00:00:00|      11947|    CANCELED|
|     753|2013-07-29 00:00:00|       5094|    CA

In [158]:
#sort Orderby OrderDate and orderStatus [composite Sorting]
dftyped.orderBy('order_date','order_status').show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|      50|2013-07-25 00:00:00|       5225|    CANCELED|
|       1|2013-07-25 00:00:00|      11599|      CLOSED|
|      12|2013-07-25 00:00:00|       1837|      CLOSED|
|       4|2013-07-25 00:00:00|       8827|      CLOSED|
|      37|2013-07-25 00:00:00|       5863|      CLOSED|
|      18|2013-07-25 00:00:00|       1205|      CLOSED|
|      24|2013-07-25 00:00:00|      11441|      CLOSED|
|      25|2013-07-25 00:00:00|       9503|      CLOSED|
|   57754|2013-07-25 00:00:00|       4648|      CLOSED|
|      90|2013-07-25 00:00:00|       9131|      CLOSED|
|      51|2013-07-25 00:00:00|      12271|      CLOSED|
|      57|2013-07-25 00:00:00|       7073|      CLOSED|
|      61|2013-07-25 00:00:00|       4791|      CLOSED|
|      62|2013-07-25 00:00:00|       9111|      CLOSED|
|      87|2013-07-25 00:00:00|       3065|      

In [159]:
#sort by order_item_order_id and order_item_sutotal in descending order
orderItemsDF.orderBy(orderItemsDF.order_item_order_id,orderItemsDF.order_item_subtotal.desc()).show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            6|                  4|                  365|                  5|             299.95|                   59.99|
|            8| 

In [160]:
#writing Df to storage.
#orderItemsDF.write.Differentoptions

In [163]:
#Aggregations, Ranking,Windowing functions- APIs
#Get Top N products per Day
#Creating Window Specs
#Performing aggregation
#using Windowing Functions
#Ranking in each Partitions/Group
#Development Life Cycle
orderItemsDF.groupBy('order_item_order_id').agg(round(sum('order_item_subtotal'),2).alias('Total Revenue')).show()
#we want to see what is the % of the revenue in total revenue

+-------------------+-------------+
|order_item_order_id|Total Revenue|
+-------------------+-------------+
|                148|       479.99|
|                463|       829.92|
|                471|       169.98|
|                496|       441.95|
|               1088|       249.97|
|               1580|       299.95|
|               1591|       439.86|
|               1645|      1509.79|
|               2366|       299.97|
|               2659|       724.91|
|               2866|       569.96|
|               3175|       209.97|
|               3749|       143.97|
|               3794|       299.95|
|               3918|       829.93|
|               3997|       579.95|
|               4101|       129.99|
|               4519|        79.98|
|               4818|       399.98|
|               4900|       179.97|
+-------------------+-------------+
only showing top 20 rows



In [220]:
PerOrderTotalRevenue = orderItemsDF.groupBy('order_item_order_id').agg(round(sum('order_item_subtotal'),2).alias('Total_Revenue'))
#PerOrderTotalRevenue.show()
#while doing groupBy(), we will be only able to get that column details only if we need other columns along with groupby then it's not suggest to use group by 
#better to go with partition clause

PerOrderRevTotalRev = PerOrderTotalRevenue.join(orderItemsDF,on='order_item_order_id').select('order_item_order_id','Total_Revenue','order_item_subtotal')
#PerOrderRevTotalRev.show()


#The same can be achieved using window function
from pyspark.sql.window import Window
spec = Window.partitionBy('order_item_order_id')
#orderItemsDF.select('order_item_order_id','order_item_subtotal',round(sum(orderItemsDF.order_item_subtotal).over(spec),2).alias('totalRevneue')).show()
df2 = orderItemsDF.select('order_item_order_id','order_item_subtotal', sum(orderItemsDF.order_item_subtotal).over(spec).alias('totalRevneue'))
df2.select('order_item_order_id','order_item_subtotal','totalRevneue',round((df2.order_item_subtotal*100)/(df2.totalRevneue),2).alias('%')).show()

+-------------------+-------------------+------------------+-----+
|order_item_order_id|order_item_subtotal|      totalRevneue|    %|
+-------------------+-------------------+------------------+-----+
|                148|              100.0|479.99000549316406|20.83|
|                148|              250.0|479.99000549316406|52.08|
|                148|             129.99|479.99000549316406|27.08|
|                463|             239.96| 829.9200096130371|28.91|
|                463|              250.0| 829.9200096130371|30.12|
|                463|              39.99| 829.9200096130371| 4.82|
|                463|             299.97| 829.9200096130371|36.14|
|                471|              39.99|169.98000717163086|23.53|
|                471|             129.99|169.98000717163086|76.47|
|                496|              59.99|  441.950008392334|13.57|
|                496|              150.0|  441.950008392334|33.94|
|                496|              51.99|  441.950008392334|11

In [222]:
#PerOrderRevTotalRev.show()
#PerOrderTotalRevenue.select('order_item_order_id','Total_Revenue','order_item_subtotal').show()

In [228]:
groupById = Window.partitionBy('order_item_order_id')
#orderItemsDF.withColumn('order_item_subtotal',sum('order_item_subtotal').over(groupById)).show()
orderItemsDF.select('order_item_subtotal',sum('order_item_subtotal').over(groupById).alias('revenue'), avg('order_item_subtotal').over(groupById)).show()

+-------------------+------------------+------------------------------------------------------------------------------------+
|order_item_subtotal|           revenue|avg(order_item_subtotal) OVER (PARTITION BY order_item_order_id unspecifiedframe$())|
+-------------------+------------------+------------------------------------------------------------------------------------+
|              100.0|479.99000549316406|                                                                  159.99666849772134|
|              250.0|479.99000549316406|                                                                  159.99666849772134|
|             129.99|479.99000549316406|                                                                  159.99666849772134|
|             239.96| 829.9200096130371|                                                                  207.48000240325928|
|              250.0| 829.9200096130371|                                                                  207.48000240

In [240]:
# get Average Revenue of each day and get all the orders who earn more than avg revenue
# orderDF.show()
# orderItemsDF.show()
orderDFNew = orderDF.filter((orderDF.order_status == 'COMPLETE').__or__(orderDF.order_status=='CLOSED'))
#orderDFNew.show()
joinOrder_OrderItems = orderDFNew.join(orderItemsDF, orderDF.order_id == orderItemsDF.order_item_order_id).select('order_id','order_date','order_item_subtotal')
#joinOrder_OrderItems.show()

spec = Window.partitionBy('order_date')
avgPerDay = joinOrder_OrderItems.select('order_id',avg('order_item_subtotal').over(spec))
avgPerDay.show()

+--------+---------------------------------------------------------------------------+
|order_id|avg(order_item_subtotal) OVER (PARTITION BY order_date unspecifiedframe$())|
+--------+---------------------------------------------------------------------------+
|   14232|                                                          205.6111187008645|
|   14232|                                                          205.6111187008645|
|   14233|                                                          205.6111187008645|
|   14235|                                                          205.6111187008645|
|   14236|                                                          205.6111187008645|
|   14236|                                                          205.6111187008645|
|   14236|                                                          205.6111187008645|
|   14237|                                                          205.6111187008645|
|   14237|                                 